In [3]:
!pip install --upgrade numpyro


     |████████████████████████████████| 176 kB 4.4 MB/s 
     |████████████████████████████████| 42.0 MB 13.9 MB/s 
     |████████████████████████████████| 473 kB 38.1 MB/s 
  Created wheel for jax: filename=jax-0.2.3-py3-none-any.whl size=542175 sha256=2dd8f7189ef66f1144660aaee29e50ed812a68c19acda1033d53d0bd1a78c211
  Stored in directory: /Users/alexkotlar/Library/Caches/pip/wheels/93/6f/2d/ee26ee4ada4c80f15c60eacb11a410d20f59e244bfea506111
Successfully built jax
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.55
    Uninstalling jaxlib-0.1.55:
      Successfully uninstalled jaxlib-0.1.55
  Attempting uninstall: jax
    Found existing installation: jax 0.2.0
    Uninstalling jax-0.2.0:
      Successfully uninstalled jax-0.2.0
  Attempting uninstall: numpyro
    Found existing installation: numpyro 0.4.0
    Uninstalling numpyro-0.4.0:
      Successfully uninstalled numpyro-0.4.0


In [8]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, Gamma, Categorical, Uniform
from jax import random
from numpyro.infer import MCMC, NUTS
import numpy as np

In [1]:
from mvl import genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([2., 2.]), afMean = tensor(5e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"] #TODO: normalized names, and indicate that the params pDs are incomplete

covShared tensor([[1.0000, 0.9500],
        [0.9500, 1.0000]])
residualCovariance tensor([[1, 0],
        [0, 1]])
PD1 threshold, PD2 threshold tensor(-2.3263) tensor(-2.3263)
pDsWithBoth tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)
pdThresh tensor([2.3263, 2.3263])
pdTarget tensor([0.0200, 0.0200])
pdvthresh tensor([2.0537, 2.0537])
meanEffect tensor([0.2726, 0.2726])
meanEffectsAcrossAllGenes tensor([0.2726, 0.2726])
allEffects tensor([[-0.2243, -0.2284],
        [-0.4096, -0.3958],
        [-0.2490, -0.1867],
        ...,
        [-0.2296, -0.2595],
        [-0.3662, -0.3316],
        [-0.3309, -0.2820]])
PD1GivenV.mean() tensor(0.0205) PD2GivenV.mean() tensor(0.0205)
allEffects[i] tensor([-0.2243, -0.2284])
PDBothGivenV.mean tensor(0.0143, dtype=torch.float64)
PDBothGivenV / PDBoth tensor(2.1377, dtype=torch.float64)
pdsCovarOnMean.mean(0) tensor([0.0205, 0.0205, 0.0143], dtype=torch.float64)
np.corrcoef(pdvInBoth[:,0], pdvInBoth[:,1])
 [[1.         0.94835334]
 [0.9483533

In [21]:
def mix_weights(beta):
    beta1m_cumprod = (1 - beta).cumprod(-1)
    return np.pad(beta, (0, 1), constant_values=1) * np.pad(beta1m_cumprod, (1, 0), constant_values=1)
    

In [27]:
import torch.nn.functional as F
def mix_weights_torch(beta):
    beta1m_cumprod = (1 - beta).cumprod(-1)
    return F.pad(beta, (0, 1), value=1) * F.pad(beta1m_cumprod, (1, 0), value=1)
    

In [30]:
mix_weights(np.array([10,20]))

array([  10, -180,  171])

In [31]:
mix_weights_torch(np.array([10,20]))

AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [40]:
import torch
t4d = torch.empty(3, 3, 4, 2)
p1d = (1, 1)
out = F.pad(t4d, p1d, "constant", 0)
out
F.pad?

Signature: F.pad(input, pad, mode='constant', value=0)
Docstring:
Pads tensor.

Padding size:
    The padding size by which to pad some dimensions of :attr:`input`
    are described starting from the last dimension and moving forward.
    :math:`\left\lfloor\frac{\text{len(pad)}}{2}\right\rfloor` dimensions
    of ``input`` will be padded.
    For example, to pad only the last dimension of the input tensor, then
    :attr:`pad` has the form
    :math:`(\text{padding\_left}, \text{padding\_right})`;
    to pad the last 2 dimensions of the input tensor, then use
    :math:`(\text{padding\_left}, \text{padding\_right},`
    :math:`\text{padding\_top}, \text{padding\_bottom})`;
    to pad the last 3 dimensions, use
    :math:`(\text{padding\_left}, \text{padding\_right},`
    :math:`\text{padding\_top}, \text{padding\_bottom}`
    :math:`\text{padding\_front}, \text{padding\_back})`.

Padding mode:
    See :class:`torch.nn.ConstantPad2d`, :class:`torch.nn.ReflectionPad2d`, and
    :class:`

In [39]:
import torch
t4d = np.empty([3, 3, 4, 2])
p1d = (1, 1)
out = np.pad(t4d, p1d, mode="constant", constant_values=0)
out

array([[[[ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],
         [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],
         [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],
         [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],
         [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000]],

        [[ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],
         [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],
         [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],
         [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
           0.00000000e+000,  0.00000000e+000],

In [19]:

nHypotheses = 4
kConditions = 4
altCounts = liabData55cov["altCounts"].numpy()
N = len(liabData55cov["altCounts"])

nCases = liabParams55cov["nCases"].numpy()
nCtrls = liabParams55cov["nCtrls"].numpy()
samplePDs = nCases / (nCases.sum() + nCtrls)
pdsAll = tensor([1 - samplePDs.sum(), *samplePDs])
print("pdsAll", pdsAll)
# pdsAllPop = tensor([1 - pDs.sum(), *pDs])
# print("pdsall", pdsAllPop)
# pdsAll = pdsAllPop
pdsAllShaped = pdsAll.expand(nHypotheses, kConditions)

# tfpdsAllShaped = tf.convert_to_tensor(pdsAllShaped)
# total_count = liabData55cov["altCounts"].sum(1)
# print(total_count.shape, total_count)
alpha = .05
def model(data):
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        # Alex: this could maybe be fixed parameters, shared across all observations with different patterns
        # in which case it's not needed
        # or could we patterned independently per cluster as raymond had suggested
        probs = numpyro.sample("probs", Dirichlet(np.ones(kConditions)))
        # print("probs")

    with numpyro.plate("data", N):
        print('beta', beta)
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        # print("probs", probs[z].shape, probs[z])
        return numpyro.sample("obs", Multinomial(probs=probs[z]*pdsAll, validate_args=False), obs=data)
    
# def guide(data):
#     alphas = numpyro.param("alphas", lambda: Gamma(np.ones(kConditions), np.ones(kConditions)).sample())
#     kappa = numpyro.param('kappa', lambda: Uniform(0, 2).sample([T-1]), constraint=constraints.positive)
#     # tau = pyro.param('tau', lambda: MultivariateNormal(torch.zeros(2), 3 * torch.eye(2)).sample([T]))
#     phi = numpyro.param('phi', lambda: Dirichlet(1/nHypotheses * np.ones(nHypotheses)).sample([N]), constraint=constraints.simplex)
#     # print("alphas", alphas)
#     with numpyro.plate("beta_plate", nHypotheses-1):
#         q_beta = numpyro.sample("beta", Beta(np.ones(T-1), kappa))

#     with numpyro.plate("mu_plate", nHypotheses):
#         # alphas = alphas.detach().numpy()
#         # print("alphas", alphas[0])
#         aShaped = np.array([
#             [alphas[0], alphas[0], alphas[0], alphas[0]],
#             [alphas[0], alphas[1], alphas[0], alphas[1]],
#             [alphas[0], alphas[0], alphas[2], alphas[2]],
#             alphas
#         ])
#         # print("aShaped", aShaped)
#         probs = numpyro.sample("probs", Dirichlet(aShaped))
#         # print("probs.shape", probs.shape)

#     with numpyro.plate("data", N):
#         z = numpyro.sample("z", Categorical(phi))

pdsAll tensor([0.5952, 0.1786, 0.1786, 0.0476], dtype=torch.float64)


In [20]:
def main(_args):
    # data = generate_data()
    nuts = NUTS(model)
    init_rng_key = random.PRNGKey(1273)
    mcmc = MCMC(nuts, 100, 1000)
    res = mcmc.run(init_rng_key, altCounts)
    mcmc.print_summary()

main(None)

ValueError: unsupported keyword arguments for mode 'constant': {'value'}